In [ ]:
%matplotlib widget
import ibvpy.api as ib
import matplotlib.pylab as plt
from matplotlib import cm
from bmcs_shear.dic_crack import\
    DICGrid, DICStateFields, DICCrackList, \
    DICAlignedGrid
from bmcs_shear.api import DICInpUnstructuredPoints
import numpy as np
np.seterr(divide ='ignore', invalid='ignore');
import collections
collections.Iterable = collections.abc.Iterable
from bmcs_shear.dic_crack import DICStressProfile

In [ ]:
tests = {'B9_TV2': [1, 7], 
         'B1_TV1': [4]}

phi_M_list = []

for test, C_range in tests.items():
    dic_grid = DICGridTri(U_factor=100, dir_name=test, t=1, padding=40, d_x=3, d_y=3,
                         T_stepping='delta_T', n_T_max = 40)
    dic_grid.read_beam_design()
    dic_grid.trait_set(d_x=5, d_y=5);
    dsf = DICStateFields(dic_grid=dic_grid, R=10, omega_threshold=0.2, verbose_eval=True)
    dsf.tmodel_.trait_set(E=1400, c_T=0.006, nu=0.18, epsilon_0=0.0018,
                          epsilon_f=0.002, eps_max=0.01);
    dsf.eval()
    dcl = DICCrackList(dsf=dsf, corridor_left=4, corridor_right=1, x_boundary = 15)
    dcl.identify_cracks()
    
    n_C = len(dcl.items)
    t_range = np.linspace(0, 1, dic_grid.n_T) 
    for C in C_range:
        print('crack', C)
        phi_M_ = []
        cor = dcl.items[str(C)].cor
        for t in t_range: 
            cor.dic_grid.t = t
            phi_M_.append([cor.phi_t, cor.M_t])
        phi_M_list.append(np.array(phi_M_))

In [ ]:
# np.savez_compressed('phi_M_C', phi_M_C=phi_M_C)

In [ ]:
# loaded = np.load('phi_M_C.npz')

In [ ]:
%matplotlib widget
fig, ax = plt.subplots(1,1)
colors = ['red', 'blue', 'purple', 'orange', 'green', 'black', 'magenta']
for C, (phi_M, color) in enumerate(zip(phi_M_list, colors)):
    ax.plot(*phi_M.T, color=color, label=str(C))
ax.legend()
# ax.plot(*M_phi_C[0].T, color='orange')
# ax.plot(*M_phi_C[1].T, color='purple')
# ax.plot(*phi_M_C.T)